In [1]:
import selenium
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import requests
import re

In [2]:
driver=webdriver.Chrome(r"C:\Program Files\chromedriver.exe")

1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [3]:
driver.get("http://www.amazon.in/")

In [4]:
user_input = input('Enter the product that we want to search : ')

Enter the product that we want to search : Laptop


In [5]:
search=driver.find_element(By.XPATH,'//input[@id="twotabsearchtextbox"]')
search

<selenium.webdriver.remote.webelement.WebElement (session="1d73e4e80e7a7c30a297d87a5c979f3d", element="0043f94f-603e-446d-bec1-1b2fe9eaffdc")>

In [6]:
search.send_keys(user_input)

In [7]:
search_btn = driver.find_element(By.XPATH,'//input[@id="nav-search-submit-button"]')
search_btn.click()

2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [8]:
urls=[]
for i in range(0,3):
    Page_urls=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    for i in Page_urls:
        urls.append(i.get_attribute('href'))

    nxt_btn=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]') 
    url=nxt_btn.get_attribute('href')
    driver.get(url)
    time.sleep(2)

In [9]:
urls

['https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo0MjczMDEzOTQ5MjEzMzY0OjE2NjY5NDkwMzI6c3BfYXRmOjIwMDc0MTE5NzIzMzk4OjowOjo&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&url=%2FAcer-Aspire-Intel-Core-11th%2Fdp%2FB0B5GDPSXS%2Fref%3Dsr_1_1_sspa%3Fcrid%3D1I8WJT6LJPKWY%26keywords%3DLaptop%26qid%3D1666949032%26qu%3DeyJxc2MiOiI5LjIxIiwicXNhIjoiOS40OCIsInFzcCI6IjYuODEifQ%253D%253D%26sprefix%3D%252Caps%252C169%26sr%3D8-1-spons%26psc%3D1',
 'https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo0MjczMDEzOTQ5MjEzMzY0OjE2NjY5NDkwMzI6c3BfYXRmOjIwMDgwMzczMzgyOTk4OjowOjo&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&url=%2FHP-15-6-inch-Micro-Edge-Anti-Glare-15s-fr2511TU%2Fdp%2FB0B4N1FRW8%2Fref%3Dsr_1_2_sspa%3Fcrid%3D1I8WJT6LJPKWY%26keywords%3DLaptop%26qid%3D1666949032%26qu%3DeyJxc2MiOiI5LjIxIiwicXNhIjoiOS40OCIsInFzcCI6IjYuODEifQ%253D%253D%26sprefix%3D%252Caps%252C169%26sr%3D8-2-spons%26psc%3D1',
 'https://www.amazon.in/Lenovo-IdeaPad-Warranty-Platinum-81WQ00MQIN/dp/B09YJ1Q16L/ref=sr_1_3?crid=1I8WJT6LJPKWY&keywords=Laptop&qid=1666949032

In [10]:
len(urls)

78

In [11]:
Brand_name= []
Product_name = []
Price = []
Return = []
Expected_Delivery = []
Availability = []

In [12]:
for i in urls:
    driver.get(i)
    time.sleep(2)
    
    try:
        brand=driver.find_element(By.XPATH,'//div[@id="poExpander"]/div[1]/div/table/tbody/tr[1]/td[2]/span')
        Brand_name.append(brand.text)
    except NoSuchElementException as e:
        Brand_name.append("-")
        
    try:
        prod=driver.find_element(By.XPATH,'//div[@id="poExpander"]/div[1]/div/table/tbody/tr[2]/td[2]/span')
        Product_name.append(prod.text)
    except NoSuchElementException as e:
        Product_name.append("-")
        
    try:
        pri=driver.find_element(By.XPATH,'//div[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span[2]/span[2]/span[2]')
        Price.append(pri.text)
    except NoSuchElementException as e:
        Price.append("-")
        
    try:
        ret=driver.find_element(By.XPATH,'//div[@id="RETURNS_POLICY"]/span/div[2]/a')
        Return.append(ret.text)
    except NoSuchElementException as e:
        Return.append("-")
        
    try:
        delivary=driver.find_element(By.XPATH,'//div[@id="mir-layout-DELIVERY_BLOCK-slot-PRIMARY_DELIVERY_MESSAGE_LARGE"]/span/span')
        Expected_Delivery.append(delivary.text)
    except NoSuchElementException as e:
        Expected_Delivery.append("-")
        
    try:
        avai=driver.find_element(By.XPATH,'//div[@id="availability"]/span')
        Availability.append(avai.text)
    except NoSuchElementException as e:
        Availability.append("-")
        
Laptop=pd.DataFrame({})
Laptop['Brand Name'] = Brand_name
Laptop['Product Name'] = Product_name
Laptop['Price'] = Price
Laptop['Return'] = Return
Laptop['Expected_Delivery'] = Expected_Delivery
Laptop['Availability'] = Availability
Laptop['Product URL'] = urls

In [13]:
Laptop

,Brand Name,Product Name,Price,Return,Expected_Delivery,Availability,Product URL
0,Acer,Aspire 3,"47,990",7 Days Replacement,"Sunday, 30 October",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,HP,15s-fr2511TU,"43,990",7 Days Replacement,"Saturday, 5 November",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Lenovo,IdeaPad 3 15IGL05,"29,516",7 Days Replacement,"Sunday, 30 October",In stock.,https://www.amazon.in/Lenovo-IdeaPad-Warranty-...
3,ASUS,VivoBook 15 (2021),"25,990",7 Days Replacement,"Sunday, 30 October",In stock.,https://www.amazon.in/ASUS-15-6-inch-Integrate...
4,HP,15s-ey1508AU,"29,990",7 Days Replacement,"Thursday, 3 November",In stock.,https://www.amazon.in/HP-15s-Ryzen-15-6inch-Mi...
...,...,...,...,...,...,...,...
73,Acer,Aspire 3,"37,990",7 Days Replacement,"Monday, 31 October",In stock.,https://www.amazon.in/Acer-Aspire-Ryzen-Proces...
74,HP,HP 255 G8,"27,950",7 Days Replacement,"Sunday, 30 October",In stock.,https://www.amazon.in/HP-Laptop-3K9U1PA-Window...
75,MI,Mi Notebook Pro,"54,990",7 Days Replacement,"Sunday, 30 October",In stock.,https://www.amazon.in/Notebook-Display-i5-1130...
76,Dell,Inspiron 3511,"42,500",7 Days Replacement,"Sunday, 30 October",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [14]:
Laptop.to_csv('Amazon_{}.csv'.format(user_input))

3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [93]:
driver = webdriver.Chrome("chromedriver.exe")

url = "https://images.google.com/"

urls = []
data = []
search_item = ["fruits", "cars", "Machine Learning", "Guitar", "Cakes"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
   
    search_bar = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input') 
    
    
    search_bar.send_keys(str(item))
    
    
    search_button =driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button/div/span').click()
    
   
    for _ in range(100):
        driver.execute_script("window.scrollBy(0,100)")
        
        imgs = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:100]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response = requests.get(urls[i])

    file = open(r"C:\Users\Angelluv\OneDrive\Desktop\GoogleImages"+str(i)+".jpg", "wb")

    file.write(response.content)

4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [682]:
driver = webdriver.Chrome('chromedriver.exe')

In [683]:
url = 'https://www.flipkart.com/'
driver.get(url)

In [684]:
login_X_btn = driver.find_element(By.XPATH,'//div[@class="_2QfC02"]//button').click()

In [685]:
search_bar = driver.find_element(By.XPATH,'//input[@class="_3704LK"]')
search_bar.send_keys("pixel 4A")

In [686]:
search_btn = driver.find_element(By.XPATH,'//button[@class="L0Z3Pu"]')
search_btn.click()

In [687]:
page1_url = []
urls = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for url in urls:
    page1_url.append(url.get_attribute('href'))

In [688]:
Smartphones = {}
Smartphones["Brand Name"] = []
Smartphones["Smartphone Name"] = []
Smartphones["Colour"] = []
Smartphones["RAM"] = []
Smartphones["Storage(ROM)"] = []
Smartphones["Primary Camera"] = []
Smartphones["Secondary Camera"] = []
Smartphones["Display Size"] = []
Smartphones["Battery Capacity"] = []
Smartphones["Price"] = []
Smartphones["Product URL"] = []

In [689]:
for url in page1_url:
    driver.get(url)                                                        
    print("Scraping URL = ", url)
    Smartphones['Product URL'].append(url)                                                          
    time.sleep(2)
    
    try:
        read_more = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    
    #Scraping brand name of smartphone
    try:
        brand_tags = driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')      
        Smartphones["Brand Name"].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand Name'].append('-')
    
    
    #Scraping name of smartphone
    try:
        name_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')     
        Smartphones['Smartphone Name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Smartphone Name'].append('-')
    
    
    #Scraping colour of smartphone
    try:
        color_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
     
    
    #Scraping RAM data of smartphone
    try:
        ram_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
    
    
    #Scraping ROM data of smartphone
    try:
        rom_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        Smartphones['Storage(ROM)'].append(rom_tags.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
        
    #Scraping Primary camera data of smartphone
    try:                                                                                    
        pri_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Smartphones['Primary Camera'].append(pri_tags.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
        
    #Scraping secondary camera data of smarphone
    try:                                                                                    
        sec_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
        
    #Scraping Display size data of smartpone
    try:
        disp_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
    
   
    #Scraping battery capacity of smartphone
    try:
        bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
        if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
        bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
         
    #Scraping Price of smartphone
    try:
        price_tags = driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
        Smartphones['Price'].append('-')

Scraping URL =  https://www.flipkart.com/google-pixel-4a-just-black-128-gb/p/itm023b9677aa45d?pid=MOBFUSBNAZGY7HQU&lid=LSTMOBFUSBNAZGY7HQUWHTF0C&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=de4f92c8-384d-4ec3-b16e-7c9112a8dd92.MOBFUSBNAZGY7HQU.SEARCH&ppt=hp&ppn=homepage&ssid=kzgcw8a75s0000001666981004323&qH=9b26a23b2cff510d
Scraping URL =  https://www.flipkart.com/realme-c30-bamboo-green-32-gb/p/itmfcf69b2db5059?pid=MOBGF37G7FRK8FBJ&lid=LSTMOBGF37G7FRK8FBJLNLFYW&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=de4f92c8-384d-4ec3-b16e-7c9112a8dd92.MOBGF37G7FRK8FBJ.SEARCH&ppt=hp&ppn=homepage&ssid=kzgcw8a75s0000001666981004323&qH=9b26a23b2cff510d
Scraping URL =  https://www.flipkart.com/realme-c30-denim-black-32-gb/p/itmfcf69b2db5059?pid=MOBGF37G8TGKHFWJ&lid=LSTMOBGF37G8TGKHFWJIST95C&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_3&otracker=search&otracker1=sea

In [690]:
print(len(Smartphones["Brand Name"]),len(Smartphones["Smartphone Name"]),len(Smartphones["Colour"]),len(Smartphones["RAM"]),len(Smartphones["Storage(ROM)"]),len(Smartphones["Primary Camera"]),len(Smartphones["Secondary Camera"]),len(Smartphones["Display Size"]),len(Smartphones["Battery Capacity"]),len(Smartphones["Price"]))

24 24 24 24 24 24 24 24 24 24


In [691]:
df = pd.DataFrame.from_dict(Smartphones)
df

,Brand Name,Smartphone Name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,Product URL
0,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),-,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,realme,C30,Bamboo Green,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹6,849",https://www.flipkart.com/realme-c30-bamboo-gre...
2,realme,C30,Denim Black,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹6,849",https://www.flipkart.com/realme-c30-denim-blac...
3,realme,C30,Lake Blue,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹6,849",https://www.flipkart.com/realme-c30-lake-blue-...
4,realme,C30,Bamboo Green,3 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹6,499",https://www.flipkart.com/realme-c30-bamboo-gre...
5,realme,C30,Lake Blue,3 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹6,499",https://www.flipkart.com/realme-c30-lake-blue-...
6,realme,C30,Denim Black,3 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹6,499",https://www.flipkart.com/realme-c30-denim-blac...
7,vivo,T1 44W,Ice Dawn,4 GB,128 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),5000 mAh,"₹14,499",https://www.flipkart.com/vivo-t1-44w-ice-dawn-...
8,vivo,T1 44W,Starry Sky,4 GB,128 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),5000 mAh,"₹14,499",https://www.flipkart.com/vivo-t1-44w-starry-sk...
9,vivo,T1 44W,Starry Sky,6 GB,128 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),5000 mAh,"₹15,999",https://www.flipkart.com/vivo-t1-44w-starry-sk...


In [692]:
df.to_csv('Flipkart_{}.csv'.format(item))

5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [189]:
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening google maps web page
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)

#Sending keyword for seach bar and search button
city = input('Enter City name that has to be searched : ')
search_bar = driver.find_element(By.ID,"searchboxinput")                       
search_bar.clear()                                                             
time.sleep(2)
search_bar.send_keys(city)                                                     
search_btn = driver.find_element(By.ID,"searchbox-searchbutton")              
search_btn.click()                                                             
time.sleep(3)

try:
    url_str = driver.current_url
    print("URL Extracted: ", url_str)
    latitude_longitude = re.findall(r'@(.*)data',url_str)
    if len(latitude_longitude):
        lat_lng_list = latitude_longitude[0].split(",")
        if len(lat_lng_list)>=2:
            latitude = lat_lng_list[0]
            longitude = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(latitude, longitude))

except Exception as e:
        print("Error: ", str(e))

Enter City name that has to be searched : Mumbai
URL Extracted:  https://www.google.co.in/maps/place/Mumbai,+Maharashtra/@19.082039,72.6009708,10z/data=!3m1!4b1!4m5!3m4!1s0x3be7c6306644edc1:0x5da4ed8f8d648c69!8m2!3d19.0759837!4d72.8776559
Latitude = 19.082039, Longitude = 72.6009708


6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [321]:
driver=webdriver.Chrome("chromedriver.exe") 
driver.get('https://trak.in/')

In [322]:
fund_button = driver.find_element(By.XPATH,'//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fund_button)

In [323]:
fund_deals = {}
fund_deals['Date'] = []
fund_deals['Startup Name'] = []
fund_deals['Industry OR Vertical'] = []
fund_deals['Sub-Vertical'] = []
fund_deals['Location'] = []
fund_deals['Investor Name'] = []
fund_deals['Investment Type'] = []
fund_deals['Amount(in USD)'] = []

In [328]:
for i in range(54,56):
    # Date
    dt = driver.find_elements(By.XPATH,'//table[@id="tablepress-54"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_deals['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements(By.XPATH,'//table[@id="tablepress-54"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_deals['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements(By.XPATH,'//table[@id="tablepress-54"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_deals['Industry OR Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements(By.XPATH,'//table[@id="tablepress-54"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_deals['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements(By.XPATH,'//table[@id="tablepress-54"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_deals['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements(By.XPATH,'//table[@id="tablepress-54"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_deals['Investor Name'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements(By.XPATH,'//table[@id="tablepress-54"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_deals['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements(By.XPATH,'//table[@id="tablepress-54"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_deals['Amount(in USD)'].append(a.text)

In [329]:
fund_df = pd.DataFrame(fund_deals)
fund_df

,Date,Startup Name,Industry OR Vertical,Sub-Vertical,Location,Investor Name,Investment Type,Amount(in USD)
0,04/03/2021,DealShare,E-commerce,Online shopping platform,"Jaipur, Rajasthan",Innoven Capital,Debt Financing,"250,000,000"
1,31/03/2021,Uniphore,Technology,Conversational Service Automation (CSA),Palo Alto,Sorenson Capital Partners,Series D,"140,000,000"
2,30/03/2021,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,Krishtal Advisors Pte Ltd,Series E,"8,000,000"
3,30/03/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series F,"460,000,000"
4,23/03/2021,SkilloVilla,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"Titan Capital, others",Seed,"300,000,000"
5,25/03/2021,CityMall,E-commerce,Social ecommerce and online grocery platform,Gurgaon,Accel Partners,Series A,"11,000,000"
6,26/03/2021,DotPe,FinTech,Commerce and payments platform to offline ente...,Gurgaon,PayU,Series A,"27,500,000"
7,04/03/2021,DealShare,E-commerce,Online shopping platform,"Jaipur, Rajasthan",Innoven Capital,Debt Financing,"250,000,000"
8,31/03/2021,Uniphore,Technology,Conversational Service Automation (CSA),Palo Alto,Sorenson Capital Partners,Series D,"140,000,000"
9,30/03/2021,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,Krishtal Advisors Pte Ltd,Series E,"8,000,000"


In [330]:
fund_df.to_csv("Trak_in.csv")

7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [405]:
driver=webdriver.Chrome("chromedriver.exe")

In [406]:
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

In [407]:
best_gam_laptops = driver.find_element(By.XPATH,"//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

In [408]:
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []

In [409]:
laptop_name= driver.find_elements(By.XPATH,'//div[@class="right-container"]/div/div/div/a/h3')
for name in laptop_name:
    Laptop_Name.append(name.text)
    
    
#Scraping the data of operating system
try:
    op_sys = driver.find_elements(By.XPATH,'//div[@class="product-detail"]/div/ul/li[1]/div/div')
    for os in op_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


#Scraping data of display of the laptop
try:
    display= driver.find_elements(By.XPATH,'//div[@class="product-detail"]/div/ul/li[2]/div/div')
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass


#Scraping data of Processor
try:
    processor = driver.find_elements(By.XPATH,'//div[@class="product-detail"]/div/ul/li[4]/div/div')
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass

#Scraping data of memory
try:
    memory = driver.find_elements(By.XPATH,'//div[@class="product-detail"]/div/ul/li[5]/div/div')
    for memo in memory:
        Memory.append(memo.text)
except NoSuchElementException:
    pass

            
#Scraping data of weight
try:
    weight = driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]//tr[8]/td[3]')
    for wgt in weight:
        Weight.append(wgt.text)
except NoSuchElementException:
    pass


#Scraping data of dimensions
try:
    dimension = driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]//tr[8]/td[3]')
    for dim in dimension:
        Dimensions.append(dim.text)
except NoSuchElementException:
    pass



#Scraping data of Graph processor
try:
    graph = driver.find_elements(By.XPATH,'//div[@class="Spcs-details"][1]/table/tbody/tr[7]/td[3]')
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#Scraping data of price
try:
    price = driver.find_elements(By.XPATH,'//td[@class="smprice"]')
    for pri in price:
        Price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [410]:
print(len(Laptop_Name),len(Operating_sys),len(Display),len(Processor),len(Memory),len(Weight),len(Dimensions),len(Graph_proc),len(Price))

10 10 10 10 10 10 10 10 10


In [411]:
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = Laptop_Name
Gaming_Laptop['Operating system'] = Operating_sys
Gaming_Laptop['Display'] = Display
Gaming_Laptop['Processor'] = Processor
Gaming_Laptop['Memory'] = Memory
Gaming_Laptop['Weight'] = Weight
Gaming_Laptop['Dimensions'] = Dimensions
Gaming_Laptop['Graphical Processor'] = Graph_proc
Gaming_Laptop['Price'] = Price
Gaming_Laptop

,Laptop Name,Operating system,Display,Processor,Memory,Weight,Dimensions,Graphical Processor,Price
0,MSI Titan GT77-12UHS,Windows 11 Home OS,"17.3"" (3840 x 2160) Display",12th Gen Intel Core i9-12900HX | NA Processor,2 TB SSD/64 GB DDR5 Memory,397 x 330 x 23 mm dimension & 3.3 kg weight,397 x 330 x 23 mm dimension & 3.3 kg weight,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,N/A
1,Alienware X17 R2,Windows 11 Home OS,"17.3"" (1920 x 1080) Display",12th Gen Intel Core i9-12900H | 5 GHz Processor,1 TB SSD/32 GB DDR5 Memory,299 x 359 x 21 mm dimension & 2.96 kg weight,299 x 359 x 21 mm dimension & 2.96 kg weight,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,"Rs 389,990"
2,Acer Predator Triton 500 SE PT516-52s,Windows 11 Home OS,"16"" (2560 x 1600) Display",12th Gen Intel Core i7-12700H | 3.50 GHz Proce...,2 TB SSD/32 GB DDR5 Memory,358 x 262 x 19.9 mm dimension & 2.4 kg weight,358 x 262 x 19.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTXTM 3070 Ti Graphic...,"Rs 300,000"
3,Omen By HP (16-B1371TX),Windows 11 Home OS,"16.1"" (2560 x 1440) Display",12th Gen Intel Core i7-12700H | 4.7 GHz Processor,1 TB SSD/8 GB DDR5 Memory,369 x 248 x 23 mm dimension & 2.32 kg weight,369 x 248 x 23 mm dimension & 2.32 kg weight,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,"Rs 176,290"
4,Acer Predator Helios 300 AN515-45 (NH.QBRSI.0,Windows 11 Home OS,"15.6"" (2560 x 1440) Display",AMD Ryzen 9-5900HX | NA Processor,512 GB SSD/16 GB DDR4 Memory,363 x 255 x 23.9 mm dimension & 2.4 kg weight,363 x 255 x 23.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,"Rs 172,999"
5,MSI Delta 15 (A5EFK-083IN),Windows 11 Home OS,"15.6"" (1920 x 1080) Display",AMD 5th Gen Ryzen 9-5900HX | 3.3GHz Processor,1 TB SSD/16 GBGB DDR4 Memory,357 x 247 x 19 mm dimension & 1.9 kg weight,357 x 247 x 19 mm dimension & 1.9 kg weight,10 GB DDR6 AMD Radeon RX 6700M Graphics card,"Rs 188,990"
6,Omen By HP (16-C0141AX),Windows 11 Home OS,"16.1"" (2560 x 1440) Display",AMD Ryzen™ 9 5900HX | 3.3 GHz Processor,1 TB NVMe/16 GB DDR4 Memory,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,"Rs 124,990"
7,Lenovo Legion 5i Pro (82RF00MGIN),Windows 11 Home OS,"16"" (2560 x 1600) Display",12th Gen Intel Core i7-12700H | 2.3 GHz Processor,1 TB SSD/16 GB DDR5 Memory,360 x 264 x 20 mm dimension & 2.49 kg weight,360 x 264 x 20 mm dimension & 2.49 kg weight,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs 168,500"
8,Alienware M15 R5 Ryzen Edition Icc-C780001win,Windows 11 Home OS,"15.6"" (1920 x 1080) Display",AMD Ryzen R7-5800H | 4.40 GHz Processor,512 GB SSD/16 GB DDR4 Memory,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs 144,990"
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Windows 11 Home OS,15.6 MP | NA Display,Lenovo Ryzen 7-5800H | 3.2 GHz Processor,1 TB SSD/16 GB DDR4 Memory,356 x 252 x 16 mm dimension & 1.9 kg weight,356 x 252 x 16 mm dimension & 1.9 kg weight,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,"Rs 131,990"


In [428]:
Gaming_Laptop.to_csv("Gaming_Laptops.csv")

8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [693]:
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

url = "https://www.forbes.com/?sh=69e6b8c92254"
driver.get(url)
time.sleep(3)

In [694]:
button = driver.find_element(By.XPATH,'//div[@class="_69hVhdY4"]')
button.click()

In [698]:
#select billionaire  
bill = driver.find_element(By.XPATH,'//li[@class="TjJgrPSg cD45ib6e primary"]/div//span[1]')
bill.click()
time.sleep(1)

In [699]:
#select world billionaire  
world_bill= driver.find_element(By.XPATH,'//li[@class="TjJgrPSg _2bNo56RE secondary"]/a[1]')
world_bill.click()
time.sleep(1)

In [700]:
#Scraping required data from the web page
#Creating empty lists
Rank = []
Person_Name = []
total_net_worth = []
Age = []
citizenship = []
Source = []
industry = []


while(True):
    #Scraping the data of rank
    rank_tags= driver.find_elements(By.XPATH,'//div[@class="rank"]')
    for rank in rank_tags:
        Rank.append(rank.text)
    time.sleep(1)
    
    
    #Scraping the data of names
    name_tags= driver.find_elements(By.XPATH,'//div[@class="personName"]//div')
    for name in name_tags:
        Person_Name.append(name.text)
    time.sleep(1)
    
    
    #Scraping data of age
    age_tags= driver.find_elements(By.XPATH,'//div[@class="age"]/div')
    for age in age_tags:
        Age.append(age.text)
    time.sleep(1)
    
    
    #Scraping data of citizenship
    cit_tags= driver.find_elements(By.XPATH,'//div[@class="countryOfCitizenship"]')
    for cit in cit_tags:
        citizenship.append(cit.text)
    time.sleep(1)
    
    
    #Scraping data of source of income
    sour_tags= driver.find_elements(By.XPATH,'//div[@class="source"]')
    for sour in sour_tags:
        Source.append(sour.text)    
    time.sleep(1)
    
    
    #Scraping data of Industry
    ind_tags= driver.find_elements(By.XPATH,'//div[@class="category"]/div')
    for ind in ind_tags:
        industry.append(ind.text)
        
        
    #scraping data of net_worth of billionaire 
    net_tags= driver.find_elements(By.XPATH,'//div[@class="netWorth"]')
    for i in net_tags:
        total_net_worth.append(i.text)
        time.sleep(1)

    #Clicking on next button
    try:
        next_button = driver.find_element(By.XPATH,"//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break
        
    #Scraping data of net worth
    Net_Worth = []
    for i in range(0,len(total_net_worth),2):
        Net_Worth.append(total_net_worth[i])

In [701]:
Billionaires=pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Names'] = Person_Name
Billionaires['Net Worth'] = Net_Worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = industry
Billionaires

,Rank,Names,Net Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$9.9 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$9.9 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$9.9 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$9.8 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$9.8 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,,40,United States,Airbnb,Technology
198,197.,David Geffen,,79,United States,"movies, record labels",Media & Entertainment


9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [702]:
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(2)

In [703]:
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

In [704]:
search_bar = driver.find_element(By.XPATH,'//input[@id="search"]')
search_bar.send_keys("GOT")
time.sleep(1)

In [705]:
search_btn = driver.find_element(By.XPATH,'//button[@id="search-icon-legacy"]')  
search_btn.click()
time.sleep(1)

In [706]:
video = driver.find_element(By.XPATH,"//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
video.click()

In [707]:
for _ in range(1000):
    driver.execute_script("window.scrollBy(0,10000)")

In [708]:
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

#scrape comments
cm = driver.find_elements(By.ID,"content-text")
for i in cm:
    if i.text is None:
        comments.append("--")
    else:
        comments.append(i.text)
time.sleep(5)


# scrape time when comment was posted
tm = driver.find_elements(By.XPATH,'//*[@id="header-author"]/yt-formatted-string/a')
for i in tm:
    Time.append(i.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(5)

# scrape the comment likes
like = driver.find_elements(By.XPATH,"//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for i in like:
    Likes.append(i.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])

In [709]:
print(len(comments),len(comment_time),len(No_of_Likes))

20 10 20


In [710]:
Youtube=pd.DataFrame({})
Youtube['Comments'] = comments
Youtube['Comment_time'] = comment_time
Youtube['Comment upvotes'] = No_of_Likes
Youtube

ValueError: Length of values (10) does not match length of index (20)

10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [614]:
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(3)

# Opening the homepage of hostelworld.com
url = "https://www.hostelworld.com/"
driver.get(url)

In [619]:
#locating the location search bar
search_loc = driver.find_element(By.XPATH,'//input[@id="search-input-field"]')
search_loc.send_keys("London")
time.sleep(2)

In [620]:
#select london
london = driver.find_element(By.XPATH,'//*[@id="predicted-search-results"]/li[2]')
london.click()
time.sleep(2)

In [622]:
# do click in search button
search_btn = driver.find_element(By.XPATH,'//button[@id="search-button"]')
search_btn.click()

In [623]:
# Make empty lists
Hostel_Name = []
Distance = []
overall_review = []
total_reviews = []
facilities = []
price = []
Rating = []
property_description = []

In [672]:
while(True):
    
    # Hostel name
    names = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]')
    for name in names:
        Hostel_Name.append(name.text)
    time.sleep(2)
        
    # Distance from city centre
    dis = driver.find_elements(By.XPATH,'//span[@class="description"]')
    for d in dis:
        Distance.append(d.text)
    time.sleep(2)
        
    # Overall Review    
    review = driver.find_elements(By.XPATH,'//div[@class="keyword"]/span')
    for r in review:
        overall_review.append(r.text)
    time.sleep(2)
    
    # Total No of reviews     
    t_review = driver.find_elements(By.XPATH,'//div[@class="reviews"]')
    for tr in t_review:
        total_reviews.append(tr.text)
    time.sleep(2)
    
    # facilities
    service = driver.find_elements(By.XPATH,'//div[@class="facilities-label facilities"]')
    for s in service:
        facilities.append(s.text)
    time.sleep(2)
    
    # Prices    
    prices = driver.find_elements(By.XPATH,'//div[@class="price title-5"]')
    for p in prices:
        price.append(p.text)
    time.sleep(2)    
 
 # Separate  Privates_From price  and  Dorms_From price
    Privates_From = []
    for i in range(0,len(price),2):
        Privates_From.append(price[i])
        time.sleep(2)
        
    Dorms_From = []
    for i in range(1,len(price),2):
        Dorms_From.append(price[i])   
        
    try:
        next_button = driver.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break
        
time.sleep(4)               

In [673]:
# scrape Hostels URL
hostel_url = []

while(True):
    urls = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]//a')
    for url in urls:
        hostel_url.append(url.get_attribute("href"))
    time.sleep(2)    
        
    try:
        next_button = driver.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break


        
Rate = []
for page in hostel_url:
    driver.get(page)
    
    # Rating
    try:
        ratings = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[1]/section/div[6]/div/div[1]/div[1]/div[1]')
        Rate.append(ratings.text)
    except NoSuchElementException:
        Rate.append("No Rating")  
    time.sleep(2)
    
    
    # Property Description
    try:
        pd = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append("No Description")  

    
time.sleep(2)        
# remove extra data from Rating     
all_text = []
for i in Rate:
    all_text.append(i.split())
time.sleep(2)

for i in all_text:
    Rating.append(i[0]) 

In [645]:
df = pd.DataFrame({'Hostel Name':Hostel_Name,
                'Distance from city centre':Distance,
                'Overall Review':overall_review[:19],
                'Total Reviews':total_reviews,
                'Facilities':facilities,
                'Privates From Price':Privates_From,
                'Dorms From Price':Dorms_From,
                'Rating':Rating,
                'Property Description':property_description})
df

ValueError: All arrays must be of the same length

In [674]:
len(Hostel_Name)

20

In [675]:
len(Distance)

20

In [676]:
len(overall_review)

23

In [677]:
len(total_reviews)

22

In [678]:
len(facilities)

20

In [679]:
len(price)

22

In [680]:
len(Rating)

20

In [681]:
len(property_description)

20